In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# importing Word2Vec
from gensim.models import Word2Vec
from gensim.models import FastText

In [52]:
df=pd.read_excel('/content/drive/MyDrive/MinorProject2/BERT2/english_train_new.xlsx')
df

,tweet_id,text,task1,task2,ID
0,1123757263427186690,"hate wen females hit ah nigga with tht bro 😂😂,...",HOF,PRFN,hasoc_2020_en_2574
1,1123733301397733380,RT @airjunebug: When you're from the Bay but y...,HOF,PRFN,hasoc_2020_en_3627
2,1123734094108659712,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NOT,NONE,hasoc_2020_en_3108
3,1126951188170199049,RT @SheLoveTimothy: He ain’t on drugs he just ...,HOF,PRFN,hasoc_2020_en_3986
4,1126863510447710208,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NOT,NONE,hasoc_2020_en_5152
...,...,...,...,...,...
3703,1126887103437123584,RT @FilthyArt_: TONIGHT TONIGHT TONIGHT \n\nCa...,NOT,NONE,hasoc_2020_en_109
3704,1126825614906937344,RT @abbn0rmal_: Eat my ass,HOF,PRFN,hasoc_2020_en_2345
3705,1126880392550731776,RT @FlyTPA: BREAKING NEWS: TPA is about to get...,NOT,NONE,hasoc_2020_en_1039
3706,1130290906932891648,RT @StarrThaRapper: It’s been a hr FUCK THAT G...,HOF,PRFN,hasoc_2020_en_2817


In [53]:
X=df[['text','task2']]
X.head()

,text,task2
0,"hate wen females hit ah nigga with tht bro 😂😂,...",PRFN
1,RT @airjunebug: When you're from the Bay but y...,PRFN
2,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NONE
3,RT @SheLoveTimothy: He ain’t on drugs he just ...,PRFN
4,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NONE


In [54]:
df1=pd.read_csv('/content/drive/MyDrive/MinorProject2/BERT2/english_test_2919.tsv','\t')
df1

,text_id,text,task_1,task_2,task_3
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...,NOT,NONE,NONE
1,hasoc_en_416,68.5 million people have been forced to leave ...,NOT,NONE,NONE
2,hasoc_en_207,"You came, you saw .... we will look after the ...",NOT,NONE,NONE
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...,NOT,NONE,NONE
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...,HOF,PRFN,UNT
...,...,...,...,...,...
1148,hasoc_en1_3958,Each of the divorce must be pronounced with a ...,NOT,NONE,NONE
1149,hasoc_en1_4648,He is 62-yr-old Harswaroop Chauhan. His only s...,NOT,NONE,NONE
1150,hasoc_en1_4832,"Peace of graveyard or peace of mind, @asadowai...",NOT,NONE,NONE
1151,hasoc_en1_3721,The humble #tribute on the sacrifice day of #J...,NOT,NONE,NONE


In [55]:
Y=df1[['text','task_2']]
Y.head(20)

,text,task_2
0,West Bengal Doctor Crisis: Protesting doctors ...,NONE
1,68.5 million people have been forced to leave ...,NONE
2,"You came, you saw .... we will look after the ...",NONE
3,We'll get Brexit delivered by October 31st. ...,NONE
4,Fuck you. Go back to the dark ages you cow @IB...,PRFN
5,Boris Johnson faces Supreme Court bid to make ...,NONE
6,What about a refund for not serving Halala to ...,NONE
7,"General election, DUP dumped out, Tory power w...",NONE
8,#Repost free.wicked • • • • • • #freewicked ...,NONE
9,Jesus Christ Christian News. Illuminati is now...,HATE


In [56]:
Y.rename(columns = {'task_2':'task2'}, inplace = True)
Y 

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text,task2
0,West Bengal Doctor Crisis: Protesting doctors ...,NONE
1,68.5 million people have been forced to leave ...,NONE
2,"You came, you saw .... we will look after the ...",NONE
3,We'll get Brexit delivered by October 31st. ...,NONE
4,Fuck you. Go back to the dark ages you cow @IB...,PRFN
...,...,...
1148,Each of the divorce must be pronounced with a ...,NONE
1149,He is 62-yr-old Harswaroop Chauhan. His only s...,NONE
1150,"Peace of graveyard or peace of mind, @asadowai...",NONE
1151,The humble #tribute on the sacrifice day of #J...,NONE


In [57]:
cols=['task2']
X[cols] = X[cols].apply(lambda x: pd.factorize(x)[0] + 0)
Y[cols] = Y[cols].apply(lambda x: pd.factorize(x)[0] + 0)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [58]:
X.head()

,text,task2
0,"hate wen females hit ah nigga with tht bro 😂😂,...",0
1,RT @airjunebug: When you're from the Bay but y...,0
2,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,1
3,RT @SheLoveTimothy: He ain’t on drugs he just ...,0
4,RT @TavianJordan: Summer ‘19 I’m coming for yo...,1


In [59]:
Y.head()

,text,task2
0,West Bengal Doctor Crisis: Protesting doctors ...,0
1,68.5 million people have been forced to leave ...,0
2,"You came, you saw .... we will look after the ...",0
3,We'll get Brexit delivered by October 31st. ...,0
4,Fuck you. Go back to the dark ages you cow @IB...,1


In [60]:
print(X['task2'].value_counts())
print(Y['task2'].value_counts())



1    1852
0    1377
2     321
3     158
Name: task2, dtype: int64
0    865
2    124
1     93
3     71
Name: task2, dtype: int64


In [61]:
X_train=X['text']
X_train

0       hate wen females hit ah nigga with tht bro 😂😂,...
1       RT @airjunebug: When you're from the Bay but y...
2       RT @DonaldJTrumpJr: Dear Democrats: The Americ...
3       RT @SheLoveTimothy: He ain’t on drugs he just ...
4       RT @TavianJordan: Summer ‘19 I’m coming for yo...
                              ...                        
3703    RT @FilthyArt_: TONIGHT TONIGHT TONIGHT \n\nCa...
3704                           RT @abbn0rmal_: Eat my ass
3705    RT @FlyTPA: BREAKING NEWS: TPA is about to get...
3706    RT @StarrThaRapper: It’s been a hr FUCK THAT G...
3707               You don't know who iam but i know you😄
Name: text, Length: 3708, dtype: object

In [62]:
y_train=X['task2']
y_train

0       0
1       0
2       1
3       0
4       1
       ..
3703    1
3704    0
3705    1
3706    0
3707    1
Name: task2, Length: 3708, dtype: int64

In [63]:
X_test=Y['text']
X_test

0       West Bengal Doctor Crisis: Protesting doctors ...
1       68.5 million people have been forced to leave ...
2       You came, you saw .... we will look after the ...
3       We'll get Brexit delivered by October 31st.   ...
4       Fuck you. Go back to the dark ages you cow @IB...
                              ...                        
1148    Each of the divorce must be pronounced with a ...
1149    He is 62-yr-old Harswaroop Chauhan. His only s...
1150    Peace of graveyard or peace of mind, @asadowai...
1151    The humble #tribute on the sacrifice day of #J...
1152    Happy Birthday to Sh @nitin_gadkari Ji, God bl...
Name: text, Length: 1153, dtype: object

In [64]:
y_test=Y['task2']
y_test

0       0
1       0
2       0
3       0
4       1
       ..
1148    0
1149    0
1150    0
1151    0
1152    0
Name: task2, Length: 1153, dtype: int64

In [65]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3708,), (1153,), (3708,), (1153,))

In [66]:
def create_tokens(lines):
  #stop_words = set(nltk.corpus.stopwords.words('english'))
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  #t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  return t_line

In [67]:
X_train=create_tokens(X_train)
print(X_train)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenizer


0       [hate, wen, females, hit, ah, nigga, with, tht...
1       [RT, @, airjunebug, :, When, you, 're, from, t...
2       [RT, @, DonaldJTrumpJr, :, Dear, Democrats, :,...
3       [RT, @, SheLoveTimothy, :, He, ain, ’, t, on, ...
4       [RT, @, TavianJordan, :, Summer, ‘, 19, I, ’, ...
                              ...                        
3703    [RT, @, FilthyArt_, :, TONIGHT, TONIGHT, TONIG...
3704                 [RT, @, abbn0rmal_, :, Eat, my, ass]
3705    [RT, @, FlyTPA, :, BREAKING, NEWS, :, TPA, is,...
3706    [RT, @, StarrThaRapper, :, It, ’, s, been, a, ...
3707    [You, do, n't, know, who, iam, but, i, know, y...
Name: text, Length: 3708, dtype: object


In [68]:
length=np.average(X_train.apply(lambda x:len(x)*len(x)))
length=np.sqrt(length)
print(length)
length=32

23.32092249688501


In [69]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [70]:
word_index=tokenizer.word_index

In [71]:
X_test=create_tokens(X_test)
print(X_test)

0       [West, Bengal, Doctor, Crisis, :, Protesting, ...
1       [68.5, million, people, have, been, forced, to...
2       [You, came, ,, you, saw, ..., ., we, will, loo...
3       [We, 'll, get, Brexit, delivered, by, October,...
4       [Fuck, you, ., Go, back, to, the, dark, ages, ...
                              ...                        
1148    [Each, of, the, divorce, must, be, pronounced,...
1149    [He, is, 62-yr-old, Harswaroop, Chauhan, ., Hi...
1150    [Peace, of, graveyard, or, peace, of, mind, ,,...
1151    [The, humble, #, tribute, on, the, sacrifice, ...
1152    [Happy, Birthday, to, Sh, @, nitin_gadkari, Ji...
Name: text, Length: 1153, dtype: object


In [72]:
X_train=encode_text(tokenizer,X_train,length)
X_train
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[  185,  1534,  1223, ...,     0,     0,     0],
       [    3,     1,  3369, ...,     0,     0,     0],
       [    3,     1,  1536, ...,     0,     0,     0],
       ...,
       [    3,     1, 13484, ..., 13489,  1845,     0],
       [    3,     1, 13490, ...,     0,     0,     0],
       [    8,    47,    73, ...,     0,     0,     0]], dtype=int32)

In [73]:
X_test=encode_text(tokenizer,X_test,length)
X_test
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[ 1122, 13264,  5943, ...,     0,     0,     0],
       [ 1659,    57,    49, ...,     0,     0,     0],
       [    8,   617,     9, ...,     0,     0,     0],
       ...,
       [  794,    16,    92, ...,     0,     0,     0],
       [    5,  8103,    29, ...,     0,     0,     0],
       [  620,   363,    10, ...,     0,     0,     0]], dtype=int32)

In [74]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(np.asarray(y_test))
y_train = to_categorical(np.asarray(y_train))
print(y_test)
print(y_train)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [75]:
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3708, 32), (1153, 32), (3708, 4), (1153, 4))

In [40]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/MinorProject/crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [42]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [76]:
print((X_train))
print((X_test))
print((y_train))
print((y_test))

[[  185  1534  1223 ...     0     0     0]
 [    3     1  3369 ...     0     0     0]
 [    3     1  1536 ...     0     0     0]
 ...
 [    3     1 13484 ... 13489  1845     0]
 [    3     1 13490 ...     0     0     0]
 [    8    47    73 ...     0     0     0]]
[[ 1122 13264  5943 ...     0     0     0]
 [ 1659    57    49 ...     0     0     0]
 [    8   617     9 ...     0     0     0]
 ...
 [  794    16    92 ...     0     0     0]
 [    5  8103    29 ...     0     0     0]
 [  620   363    10 ...     0     0     0]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [77]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(13495, 300)

In [79]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Input,LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
model=Sequential()
model.add(Embedding(input_dim = 13495, output_dim = 300, weights=[embedding_matrix], input_length=length,trainable=False))
model.add(LSTM(192, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 32, 300)           4048500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 192)               378624    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 772       
Total params: 4,427,896
Trainable params: 379,396
Non-trainable params: 4,048,500
_________________________________________________________________


In [80]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=200,batch_size=256, verbose=2)

Epoch 1/200
15/15 - 2s - loss: 1.1276 - accuracy: 0.4687 - val_loss: 1.3366 - val_accuracy: 0.0807
Epoch 2/200
15/15 - 2s - loss: 1.0419 - accuracy: 0.5310 - val_loss: 1.1982 - val_accuracy: 0.4250
Epoch 3/200
15/15 - 2s - loss: 1.0124 - accuracy: 0.5574 - val_loss: 1.8202 - val_accuracy: 0.1327
Epoch 4/200
15/15 - 2s - loss: 0.8648 - accuracy: 0.6872 - val_loss: 1.9093 - val_accuracy: 0.1596
Epoch 5/200
15/15 - 2s - loss: 0.7347 - accuracy: 0.7611 - val_loss: 2.8180 - val_accuracy: 0.0486
Epoch 6/200
15/15 - 1s - loss: 0.6911 - accuracy: 0.7824 - val_loss: 2.7621 - val_accuracy: 0.0338
Epoch 7/200
15/15 - 2s - loss: 0.6566 - accuracy: 0.7940 - val_loss: 3.0378 - val_accuracy: 0.0399
Epoch 8/200
15/15 - 1s - loss: 0.6343 - accuracy: 0.8029 - val_loss: 3.3073 - val_accuracy: 0.0434
Epoch 9/200
15/15 - 1s - loss: 0.6180 - accuracy: 0.8083 - val_loss: 3.2367 - val_accuracy: 0.0416
Epoch 10/200
15/15 - 1s - loss: 0.6001 - accuracy: 0.8109 - val_loss: 3.3620 - val_accuracy: 0.0546
Epoch 11/

In [81]:
predictions_test = model.predict(X_test)

predictions_test

array([[3.9350678e-07, 9.9999952e-01, 1.2190521e-08, 1.2108032e-07],
       [1.1855584e-06, 9.9998319e-01, 1.1989421e-06, 1.4429895e-05],
       [1.8670182e-06, 9.9999797e-01, 1.6260815e-08, 5.0861235e-08],
       ...,
       [3.4823086e-05, 9.9996400e-01, 3.3267517e-07, 8.4279225e-07],
       [6.3925640e-06, 1.9601201e-01, 1.9554558e-04, 8.0378598e-01],
       [3.6128964e-05, 9.9996352e-01, 1.3764105e-07, 2.2817650e-07]],
      dtype=float32)

In [82]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df1.index)
# output_DataFrame[0]=output_DataFrame[0].map({0:'NGEN',1:'GEN'})
# output_DataFrame.to_csv("German Task2 output.csv")

In [83]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.22      0.03      0.05       865
           1       0.02      0.17      0.03        93
           2       0.39      0.10      0.15       124
           3       0.08      0.04      0.06        71

   micro avg       0.05      0.05      0.05      1153
   macro avg       0.18      0.09      0.07      1153
weighted avg       0.21      0.05      0.06      1153
 samples avg       0.05      0.05      0.05      1153



In [ ]:
  df3=df1[['tweet_id','ID']]
  df3

,tweet_id,ID
0,1127781647783301120,hasoc_2020_hi_1938
1,1127492853171535872,hasoc_2020_hi_1201
2,1123638375896903681,hasoc_2020_hi_1628
3,1127496460310683648,hasoc_2020_hi_3257
4,1127667277535399937,hasoc_2020_hi_4133
...,...,...
658,1123814066907242503,hasoc_2020_hi_466
659,1127616035689979904,hasoc_2020_hi_1758
660,1127502823044702208,hasoc_2020_hi_3521
661,1127780829914992640,hasoc_2020_hi_1293


In [ ]:
predictions_test1=predictions_test.argmax(1)
pred_test1=pd.Series(predictions_test1)
pred_test1

0      1
1      3
2      0
3      1
4      1
      ..
658    1
659    1
660    1
661    1
662    1
Length: 663, dtype: int64

In [ ]:
df3['label']=pred_test1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df3

,tweet_id,ID,label
0,1127781647783301120,hasoc_2020_hi_1938,1
1,1127492853171535872,hasoc_2020_hi_1201,3
2,1123638375896903681,hasoc_2020_hi_1628,0
3,1127496460310683648,hasoc_2020_hi_3257,1
4,1127667277535399937,hasoc_2020_hi_4133,1
...,...,...,...
658,1123814066907242503,hasoc_2020_hi_466,1
659,1127616035689979904,hasoc_2020_hi_1758,1
660,1127502823044702208,hasoc_2020_hi_3521,1
661,1127780829914992640,hasoc_2020_hi_1293,1


In [ ]:
df3['label'] = df3['label'].map({0: "NONE",1:"OFFN",2:"PRFN",3:"HATE"})
df3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,ID,label
0,1127781647783301120,hasoc_2020_hi_1938,OFFN
1,1127492853171535872,hasoc_2020_hi_1201,HATE
2,1123638375896903681,hasoc_2020_hi_1628,NONE
3,1127496460310683648,hasoc_2020_hi_3257,OFFN
4,1127667277535399937,hasoc_2020_hi_4133,OFFN
...,...,...,...
658,1123814066907242503,hasoc_2020_hi_466,OFFN
659,1127616035689979904,hasoc_2020_hi_1758,OFFN
660,1127502823044702208,hasoc_2020_hi_3521,OFFN
661,1127780829914992640,hasoc_2020_hi_1293,OFFN


In [ ]:
df3.to_csv('submission_HI_B.csv')

In [ ]:
model.save("HindiDatasetSubTask2_withLSTM_Fasttext.h5")